In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.utils import shuffle as skshuffle
import numpy as np
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
import pdb


class CNN(nn.Module):
    def __init__(self, pool='max', drop_p=0):
        super(CNN, self).__init__()
        self.drop_p = drop_p
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()

        if pool == 'max':
            self.pool = nn.MaxPool2d(kernel_size=2)
        else:
            self.pool = nn.AvgPool2d(kernel_size=2)

        if self.drop_p:
            self.drop = nn.Dropout(p=self.drop_p)
        self.fc1 = nn.Linear(64 * 28 * 23, 512)
        self.relu6 = nn.ReLU()
        self.fc2 = nn.Linear(512, 40)

        self.loss = nn.CrossEntropyLoss()

    def forward(self, x):
        if self.drop_p:
            out = self.pool(self.drop(self.relu1(self.bn1(self.cnn1(x)))))
            out = self.pool(self.drop(self.relu2(self.bn2(self.cnn2(out)))))
            out = out.view(out.size(0), -1)
            out = self.fc2(self.drop(self.relu6(self.fc1(out))))
        else:
            out = self.pool(self.relu1(self.bn1(self.cnn1(x))))
            out = self.pool(self.relu2(self.bn2(self.cnn2(out))))
            out = out.view(out.size(0), -1)
            out = self.fc2(self.relu6(self.fc1(out)))
        return out

    def cnn1_out(self, x):
        return self.cnn1(x)

    def cnn2_out(self, x):
        if self.drop_p:
            out = self.pool(self.drop(self.relu1(self.bn1(self.cnn1(x)))))
        else:
            out = self.pool(self.relu1(self.bn1(self.cnn1(x))))
        return self.cnn2(out)


class ModelEvaluator:
    def __init__(self, model, epochs, lr, batch_size, use_gpu=False, optim='adam', reg=None):
        '''
        model: instance of pytorch model class
        epochs: number of training epochs
        lr: learning rate
        use_gpu: to use gpu
        optim: optimizer used for training, SGD or adam
        '''
        self.epochs = epochs
        self.lr = lr
        self.model = model
        self.use_gpu = use_gpu
        self.train_loss = []
        self.test_loss = []
        self.test_acc = []
        self.train_iter_loss = []
        self.reg = reg
        self.batch_size = batch_size
        self.optim = optim
        if self.use_gpu:
            self.model = self.model.cuda()
        #可尝试不同优化器
        if optim == 'adam':
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        elif optim == 'sgd':
            self.optimizer = torch.optim.SGD(self.model.parameters(), lr=lr, momentum=0.9)
        elif optim == 'adadelta':
            self.optimizer = torch.optim.Adadelta(self.model.parameters(),
                                                  lr=lr, eps=1e-6,
                                                  weight_decay=0)
        elif optim == 'adagrad':
            self.optimizer = torch.optim.Adagrad(self.model.parameters(), lr=lr,
                                                 lr_decay=1e-6, weight_decay=0)
        elif optim == 'rmsprop':
            self.optimizer = torch.optim.RMSprop(self.model.parameters(), lr=lr,
                                                 alpha=0.995, eps=1e-7,
                                                 weight_decay=0)
        else:
            ValueError('Optimizer Not Supported')

    def train(self, epoch, trainloader, print_every=100):
        '''
        method for training
        '''
        self.model.train()
        loss_batch = 0
        for b_idx, (train_data, train_labels) in enumerate(trainloader):
            shape = train_data.shape
            train_data = train_data.view(shape[0], 1, shape[1], shape[2])
            if self.use_gpu:
                train_data, train_labels = train_data.cuda(), train_labels.cuda()
            # Forward Pass 
            train_preds = self.model.forward(train_data)
            loss = self.model.loss(train_preds, train_labels)
            if self.reg == 'l2':
                loss = self.regularization(loss, lam=0.0001, reg='l2')
            elif self.reg == 'l1':
                loss = self.regularization(loss, lam=0.000001, reg='l1')
            else:
                pass
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            if b_idx % print_every == 0:
                print('Train Epoch: {0} [{1}/{2} ({3:.0f}%)]\t Loss {4:.6f}'.
                      format(epoch, b_idx * len(train_data), len(trainloader.dataset),
                             100. * b_idx / len(trainloader), loss))

            train_loss = loss.item()
            self.train_iter_loss.append(train_loss)
            loss_batch += train_loss
        loss_batch /= len(trainloader)
        self.train_loss.append(loss_batch)

    def test(self, testloader):
        '''
        method for testing
        '''
        self.model.eval()
        correct_, total_ = 0, 0
        with torch.no_grad():
            loss = 0
            for test_data, test_labels in testloader:
                shape = test_data.shape
                test_data = test_data.view(shape[0], 1, shape[1], shape[2])
                if self.use_gpu:
                    test_data, test_labels = test_data.cuda(), test_labels.cuda()
                test_preds = self.model.forward(test_data)

                loss += self.model.loss(test_preds, test_labels)

                _, test_pred_labels = torch.max(test_preds.data, 1)
                total_ += test_labels.size(0)
                correct_ += (test_pred_labels.cpu() == test_labels.cpu()).sum()

            loss /= len(testloader)
            self.test_loss.append(loss)
            
            accuracy_test = torch.true_divide(100 * correct_ , total_)
            self.test_acc.append(accuracy_test)
            print('Accuracy of model on test set {0:.2f}'.format(accuracy_test))
            return accuracy_test

    def evaluator(self, trainloader, testloader, print_every=1000):
        for epoch in range(self.epochs):
            self.train(epoch, trainloader, print_every=print_every)
            if epoch % 5 == 0:
                save_model = {'epoch': epoch,
                              'state_dict': self.model.state_dict(),
                              'optimizer': self.optimizer.state_dict()}
            model_name = 'Model_lr_{}_opt_{}_epoch_{}'.format(self.lr, self.optim, epoch)
            model_dir = 'model/' +model_name
            torch.save(save_model, model_dir)
            acc_ = self.test(testloader)
        return acc_

    def regularization(self, loss, lam, reg='l2'):
        ll = torch.tensor(0.0)
        for W in self.model.parameters():
            if reg == 'l2':
                ll += W.norm(2)
            else:
                ll += W.norm(1)
        loss = loss + 0.5 * lam * ll ** 2
        return loss

    def plot_loss(self):
        '''
        to visualize loss
        '''
        plt.plot(range(len(self.train_loss)), self.train_loss, label='Training Loss')
        plt.plot(range(len(self.test_loss)), self.test_loss, label='Testing Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        return plt

    def plot_acc(self):

        plt.plot(range(len(self.test_acc)), self.test_acc, label='Testing Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        return plt

    def plot_iter_loss(self):
        '''
        to visualize loss
        '''
        plt.plot(range(len(self.train_iter_loss)), self.train_iter_loss, label='Training Loss')
        plt.xlabel('Iteration')
        plt.ylabel('Loss')
        plt.legend()
        return plt

In [2]:

from torchvision import utils
import matplotlib.pyplot as plt
import numpy as np


def vistensor(tensor, ch=0, nrow=20, padding=1):
    '''
    https://github.com/pedrodiamel/nettutorial/blob/master/pytorch/pytorch_visualization.ipynb
    '''

    n, c, w, h = tensor.shape
    if c != 3:
        tensor = tensor.view(n * c, -1, w, h)
        tensor = tensor[:, ch, :, :].unsqueeze(dim=1)

    rows = np.min((tensor.shape[0] // nrow + 1, 64))
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    plt.figure(figsize=(nrow, rows))
    plt.imshow(grid.cpu().numpy().transpose((1, 2, 0)))


def plot_img(tensor, num_cols=10):
    num_kernels = tensor.shape[0]
    num_rows = 1 + num_kernels // num_cols
    fig = plt.figure(figsize=(num_cols, num_rows))
    for i in range(num_kernels):
        ax1 = fig.add_subplot(num_rows, num_cols, i + 1)
        ax1.imshow(tensor[i], cmap='gray')
        ax1.axis('off')
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])

    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    return plt

In [3]:
import numpy as np  
import torch
import matplotlib.pyplot as plt


# load data
data = np.load('ORL_face.npz')
trainX = data['trainX']
trainY = data['trainY']
testX = data['testX']
testY = data['testY']

train_images, test_images = trainX.shape[0], testX.shape[0]
trainX = trainX.astype(np.float32).reshape(train_images, 112, 92)
testX = testX.astype(np.float32).reshape(test_images, 112, 92)
trainY = trainY.astype(np.longlong)
testY = testY.astype(np.longlong)

#trainY =torch.LongTensor(trainY)
#testY =torch.LongTensor(testY)
#print(trainY.dtype)
# Image Sanity check
sample_set = np.random.randint(0, train_images, 50)
plt = plot_img(trainX[sample_set])
plt.savefig('sanity_check_img_plot')
plt.cla()
plt.clf()



trainX = torch.from_numpy(trainX)
trainY = torch.from_numpy(trainY)
testX = torch.from_numpy(testX)
testY = torch.from_numpy(testY) 

train_data = torch.utils.data.TensorDataset(trainX, trainY)
test_data = torch.utils.data.TensorDataset(testX, testY)

batch_size = 20

train_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)


lr = 0.001
epochs = 10

drop_p = 0.3
n_in = trainX.shape[1]
n_out = len(np.unique(trainY))

pool = 'max'
optim = 'adam'
use_gpu = False

model = CNN(pool, drop_p=drop_p)
modeleval = ModelEvaluator(model, epochs, lr, batch_size, use_gpu=use_gpu, optim=optim)
acc_ = modeleval.evaluator(train_loader, test_loader, print_every=100)

modelname = 'model_optimizer_{}_lr_{}_drop_p_'.format(optim, lr)
print('Accuracy of {0} is {1:.2f}'.format(modelname, acc_))


plt = modeleval.plot_loss()
plt.savefig('train_test_loss, optim_{}_lr_{}_drop_p_{}.png'.format(optim, lr, drop_p))
plt.cla()
plt.clf()


plt = modeleval.plot_acc()
plt.savefig('test_acc, optim_{}, _lr_{}_drop_p_{}.png'.format(optim, lr, drop_p))
plt.cla()
plt.clf()

plt = modeleval.plot_iter_loss()
plt.savefig('train_iterloss, optim_{}, _lr_{}_drop_p_{}.png'.format(optim, lr, drop_p))
plt.cla()
plt.clf()



# Visualize Kernels

filters = model.modules()
model_layers = [i for i in model.children()]
first_layer = model_layers[0]
second_layer = model_layers[3]

first_kernels = first_layer.weight.data.clone()
print(first_kernels.shape)
vistensor(first_kernels)
plt.axis('off')
plt.ioff()
plt.savefig('filter conv 1 optim_{}_lr_{}_drop_p_{}.png'.format(optim, lr, drop_p))
plt.cla()
plt.clf()


second_kernels = second_layer.weight.data.clone()
print(second_kernels.shape)
vistensor(second_kernels[0, ...].reshape(1, 32, 3, 3))
plt.axis('off')
plt.ioff()
plt.savefig('filter conv 2 optim_{}_lr_{}_drop_p_{}.png'.format(optim, lr, drop_p))
plt.cla()
plt.clf()

Train Epoch: 0 [0/120 (0%)]	 Loss 3.837313
Accuracy of model on test set 7.50
Train Epoch: 1 [0/120 (0%)]	 Loss 21.578197
Accuracy of model on test set 15.83
Train Epoch: 2 [0/120 (0%)]	 Loss 13.842201
Accuracy of model on test set 43.33
Train Epoch: 3 [0/120 (0%)]	 Loss 3.742854
Accuracy of model on test set 82.50
Train Epoch: 4 [0/120 (0%)]	 Loss 1.303545
Accuracy of model on test set 95.00
Train Epoch: 5 [0/120 (0%)]	 Loss 0.653032
Accuracy of model on test set 100.00
Train Epoch: 6 [0/120 (0%)]	 Loss 0.767896
Accuracy of model on test set 99.17
Train Epoch: 7 [0/120 (0%)]	 Loss 0.269186
Accuracy of model on test set 100.00
Train Epoch: 8 [0/120 (0%)]	 Loss 0.110186
Accuracy of model on test set 100.00
Train Epoch: 9 [0/120 (0%)]	 Loss 0.130367
Accuracy of model on test set 100.00
Accuracy of model_optimizer_adam_lr_0.001_drop_p_ is 100.00
torch.Size([32, 1, 3, 3])
torch.Size([64, 32, 3, 3])


<Figure size 720x432 with 0 Axes>

<Figure size 1440x144 with 0 Axes>

<Figure size 1440x144 with 0 Axes>

In [16]:
print(train_loader)